**Connect google drive**

In [ ]:
#!rm -rf *
# Check if NVIDIA GPU is enabled
!nvidia-smi

Thu Jun  4 19:30:16 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P8    35W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
 Biercantus%20max%20zn%20verjaardag.docx
 BoardingPass.pkpass
'Colab Notebooks'
 DSC_0127.jpg
 DSC_0134.jpg
 DSC_0140.jpg
 DSC_0143.jpg
 DSC_0150.jpg
 DSC_0153.jpg
 DSC_0155.jpg
'Experiment 2.gdoc'
 jolov3
'Kasboek Menno.gsheet'
 meetrapport.gdoc
'Naamloos (1).gdoc'
'Naamloos document.gdoc'
 Naamloos.gdoc
'noname (1).gdoc'
 noname.gdoc
'OGO 15 computational biology'
'Profielwerkstuk Peru 2014.docx.gdoc'
'PWS Menno Cromwijk af reserve.docx'
 Video_201704120845

**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 13621, done.
remote: Total 13621 (delta 0), reused 0 (delta 0), pack-reused 13621
Receiving objects: 100% (13621/13621), 12.22 MiB | 16.73 MiB/s, done.
Resolving deltas: 100% (9292/9292), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:910:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1391:14: warning: unused variable ‘buff’ [-Wunused-variable]
         char buff[100];
              ^~~~
./src/image_opencv.cpp:1367:9: warning: unused variabl

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
#classes = 2 and filters = (class+5)*3
#max_batches = 2000*classes
#steps = 0.8*max_batches,0.9*max_batches
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i 's/steps=400000,450000/steps=3200,3600/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=21@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=21@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=21@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
#!echo "Plastic bottle" > data/obj.names
!echo -e 'classes= 2\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
#!wget https://pjreddie.com/media/files/darknet53.conv.74

**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/obj.zip -d data/obj

Archive:  /mydrive/yolov3/obj.zip
   creating: data/obj/can/
  inflating: data/obj/can/.DS_Store  
   creating: data/obj/__MACOSX/
   creating: data/obj/__MACOSX/can/
  inflating: data/obj/__MACOSX/can/._.DS_Store  
  inflating: data/obj/can/classes.txt  
  inflating: data/obj/__MACOSX/can/._classes.txt  
  inflating: data/obj/can/img_0_1005.jpg  
  inflating: data/obj/__MACOSX/can/._img_0_1005.jpg  
  inflating: data/obj/can/img_0_1005.txt  
  inflating: data/obj/__MACOSX/can/._img_0_1005.txt  
  inflating: data/obj/can/img_0_1024.jpg  
  inflating: data/obj/__MACOSX/can/._img_0_1024.jpg  
  inflating: data/obj/can/img_0_1024.txt  
  inflating: data/obj/__MACOSX/can/._img_0_1024.txt  
  inflating: data/obj/can/img_0_1028.jpg  
  inflating: data/obj/__MACOSX/can/._img_0_1028.jpg  
  inflating: data/obj/can/img_0_1028.txt  
  inflating: data/obj/__MACOSX/can/._img_0_1028.txt  
  inflating: data/obj/can/img_0_108.jpg  
  inflating: data/obj/__MACOSX/can/._img_0_108.jpg  
  inflating: dat

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

#txt_file_paths = glob.glob(r"data/obj/*.txt")
#for i, file_path in enumerate(txt_file_paths):
    # get image size
#    with open(file_path, "r") as f_o:
#        lines = f_o.readlines()

 #       text_converted = []
 #       for line in lines:
 #           print(line)
 #          numbers = re.findall("[0-9.]+", line)
 #           print(numbers)
 #           if numbers:

                # Define coordinates
#                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
#                text_converted.append(text)
#                print(i, file_path)
#                print(text)
        # Write file
#        with open(file_path, 'w') as fp:
#            for item in text_converted:
#                fp.writelines("%s\n" % item)

In [ ]:
import glob
images_list = glob.glob("data/obj/can/*.jpg")
print(images_list)

['data/obj/can/plastic116.jpg', 'data/obj/can/img_0_6514.jpg', 'data/obj/can/img_0_6016.jpg', 'data/obj/can/img_0_6885.jpg', 'data/obj/can/plastic71.jpg', 'data/obj/can/plastic362.jpg', 'data/obj/can/img_0_1778.jpg', 'data/obj/can/metal267.jpg', 'data/obj/can/img_0_2589.jpg', 'data/obj/can/img_0_5961.jpg', 'data/obj/can/img_0_6882.jpg', 'data/obj/can/img_0_7365.jpg', 'data/obj/can/metal280.jpg', 'data/obj/can/img_0_2715.jpg', 'data/obj/can/img_0_630.jpg', 'data/obj/can/plastic110.jpg', 'data/obj/can/metal168.jpg', 'data/obj/can/plastic346.jpg', 'data/obj/can/plastic208.jpg', 'data/obj/can/plastic94.jpg', 'data/obj/can/metal403.jpg', 'data/obj/can/img_0_2078.jpg', 'data/obj/can/plastic410.jpg', 'data/obj/can/img_0_3623.jpg', 'data/obj/can/img_0_924.jpg', 'data/obj/can/img_0_510.jpg', 'data/obj/can/img_0_1450.jpg', 'data/obj/can/plastic342.jpg', 'data/obj/can/img_0_2196.jpg', 'data/obj/can/img_0_1725.jpg', 'data/obj/can/img_0_3599.jpg', 'data/obj/can/plastic22.jpg', 'data/obj/can/img_0_6

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

In [ ]:
!rm data/obj.names
!cp data/obj/can/classes.txt data/obj.names

rm: cannot remove 'data/obj.names': No such file or directory


**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.852974, GIOU: 0.850031), Class: 0.999776, Obj: 0.969089, No Obj: 0.005283, .5R: 1.000000, .75R: 0.750000, count: 4, class_loss = 0.004054, iou_loss = 0.059742, total_loss = 0.063796 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 94 Avg (IOU: 0.927022, GIOU: 0.926178), Class: 0.999080, Obj: 0.943116, No Obj: 0.000250, .5R: 1.000000, .75R: 1.000000, count: 1, class_loss = 0.004192, iou_loss = 0.007846, total_loss = 0.012038 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 106 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000000, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.861375, GIOU: 0.855523), Class: 0.999630, Obj: 0.980030, No Obj: 0.005277, .5R: 1.000000, .75R: 1.000000, count: 3,